In [22]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [23]:
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
INDEX_NAME = "medical"

In [24]:
# Extract data from PDF
def load_pdf(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

# Create chunks
def chunk_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Execute the functions
# Note: Replace with the correct relative path to your data folder
extracted_data = load_pdf("data") 
text_chunks = chunk_documents(extracted_data)
print(f"Number of chunks: {len(text_chunks)}")

In [25]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()
print("✅ Embeddings model loaded.")

In [26]:
# If you have already populated your index using store_index.py, you can load it directly.
print("Loading vector store from existing Pinecone index...")
docsearch = PineconeVectorStore.from_existing_index(INDEX_NAME, embeddings)
print("✅ Vector store loaded.")

# Set up the retriever
retriever = docsearch.as_retriever(search_kwargs={'k': 2})

In [27]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Ensure this path is correct
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG chain created successfully.")

length of my chunk 7020


In [28]:
user_input = "What are Allergies?"
result = rag_chain.invoke(user_input)

print("Response:", result)